In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load hyperparams instance
from hyper_params import HyperParams
params = HyperParams()

In [3]:
DATA_DIR='data'

In [4]:
from dataproc import io
io.DATA_DIR='data'

In [ ]:
# build all features dataset
import featues_datasets_all_patients
featues_datasets_all_patients.DATA_DIR = DATA_DIR
df_final_dataset = featues_datasets_all_patients.run(params, bin_numerics=False)

In [40]:
df_final_dataset = None

In [ ]:
%%time
# build all features dataset for autoencoder
import featues_datasets_all_patients
featues_datasets_all_patients.DATA_DIR = DATA_DIR
df_final_dataset = featues_datasets_all_patients.run(params, binning_numerics=True, create_patients_list_view=False, create_lab_events=False )

In [231]:
df_final_dataset.shape

(54871, 1239)

In [233]:
df_lab_events = io.load_dataframe('df_lab_events')

In [235]:
df_lab_flags = featues_datasets_all_patients.pivot_flags_to_columns(df_lab_events, binning_numerics=True)

flags before binning
['50800_flag', '50801_flag', '50802_flag', '50803_flag', '50804_flag', '50805_flag', '50806_flag', '50808_flag', '50809_flag', '50810_flag', '50811_flag', '50812_flag', '50813_flag', '50814_flag', '50815_flag', '50816_flag', '50817_flag', '50818_flag', '50819_flag', '50820_flag', '50821_flag', '50822_flag', '50823_flag', '50824_flag', '50825_flag', '50826_flag', '50827_flag', '50828_flag', '50852_flag', '50853_flag', '50854_flag', '50855_flag', '50856_flag', '50857_flag', '50861_flag', '50862_flag', '50863_flag', '50864_flag', '50865_flag', '50866_flag', '50867_flag', '50868_flag', '50869_flag', '50871_flag', '50872_flag', '50873_flag', '50874_flag', '50875_flag', '50876_flag', '50877_flag', '50878_flag', '50879_flag', '50880_flag', '50881_flag', '50882_flag', '50883_flag', '50884_flag', '50885_flag', '50886_flag', '50887_flag', '50889_flag', '50890_flag', '50891_flag', '50892_flag', '50893_flag', '50894_flag', '50895_flag', '50896_flag', '50898_flag', '50899_flag'

In [226]:
df_final_dataset=df_final_dataset.drop(columns=['hadm_id'])

In [230]:
df_final_dataset.describe()

,hadm_id,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,...,Vancomycin,Vancomycin,Vancomycin 25mg/mL Ophth Soln,Vancomycin Antibiotic Lock,Vancomycin Desensitization,Vancomycin Enema,Vancomycin HCl,Vancomycin Oral Liquid,n_admits,wounds
count,54871.000000,54871.0,54871.0,54871.0,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,...,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000,54871.000000
mean,149985.025259,0.0,0.0,0.0,0.027519,0.108946,0.001112,0.061071,0.143063,0.254962,...,0.136557,0.000018,0.000073,0.000036,0.000055,0.000565,0.072096,0.009349,-0.013550,0.019136
std,28899.221980,0.0,0.0,0.0,0.163592,0.311575,0.033324,0.239462,0.350140,0.435844,...,0.343382,0.004269,0.008538,0.006037,0.007394,0.023762,0.258650,0.096239,0.090062,0.137003
min,100001.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.666667,0.000000
25%,124952.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,150005.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,175015.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,199999.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000


In [118]:
df_final_dataset = io.load_dataframe('df_final_dataset')

In [153]:
import numpy as np

In [218]:
traininng_matrix = np.load('data/autoencoder_fulldata.npy')

In [224]:
np.save('data/autoencoder_fulldata_mini.npy', traininng_matrix[:10000,:]) 

In [220]:
np.max(traininng_matrix)

1.0

In [143]:
df_final_dataset.columns.value_counts().sort_values(ascending=False)

50802_flag    1
50946_flag    1
50861_nan     1
51262_flag    1
50913_flag    1
             ..
50903_3.0     1
51244_3.0     1
51158_flag    1
50969_flag    1
50855_flag    1
Length: 1239, dtype: int64

In [165]:
stats = df_final_dataset.describe()

In [166]:
stats.loc['max'].sort_values(ascending=False)

hadm_id       199999.0
n_admits          23.0
50862_1.0          1.0
50867_3.0          1.0
50867_2.0          1.0
                ...   
51217_flag         0.0
50884_nan          0.0
51219_flag         0.0
51220_flag         0.0
51469_flag         0.0
Name: max, Length: 1239, dtype: float64

In [146]:
stats['Vancomycin']['mean']

0.13655665105429096

In [147]:
stats.loc[:, lambda df: stats[df]['mean'] > 1.0]

ValueError: Boolean array expected for the condition, not float64

In [110]:
df_dataset_unprocessed = io.load_dataframe('join_static_and_lab_data')

In [106]:
import featues_datasets_all_patients
df_dataset_unprocessed = featues_datasets_all_patients.clean_and_bin_numeric_values(df_dataset_unprocessed.reset_index(), params)

In [111]:
df_dataset_unprocessed

,50800,50800_flag,50801,50801_flag,50802,50802_flag,50803,50803_flag,50804,50804_flag,...,51529_flag,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity,gender,age
hadm_id,,,,,,,,,,,,,,,,,,,,,
100001,None,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,EMERGENCY,CLINIC REFERRAL/PREMATURE,Private,ENGL,PROTESTANT QUAKER,DIVORCED,WHITE,F,35.0
100003,ART,-1,NaN,0,-6.0,-1,NaN,0,17.0,1,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,M,59.0
100006,ART,-1,NaN,0,0.0,-1,NaN,0,24.0,-1,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,OTHER,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,F,48.0
100007,None,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Private,OTHER,JEWISH,MARRIED,WHITE,F,73.0
100009,None,0,NaN,0,2.0,-1,NaN,0,27.0,-1,...,0,EMERGENCY,TRANSFER FROM MED FACILITY,Private,OTHER,CATHOLIC,MARRIED,WHITE,M,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,ART,-1,NaN,0,1.0,-1,NaN,0,28.0,-1,...,0,EMERGENCY,TRANSFER FROM MED FACILITY,Private,OTHER,CATHOLIC,DIVORCED,UNKNOWN/NOT SPECIFIED,M,60.0
199994,ART,-1,NaN,0,-6.0,-1,NaN,0,21.0,-1,...,0,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,OTHER,CATHOLIC,SINGLE,WHITE,F,58.0
199995,None,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,EMERGENCY,PHYS REFERRAL/NORMAL DELI,Private,ENGL,PROTESTANT QUAKER,SINGLE,WHITE,M,23.0


In [52]:
import pandas as pd
pd.Series([1.0, 2.0, None]).fillna(0).apply(str).astype('O')

0    1.0
1    2.0
2    0.0
dtype: object

In [66]:
df_dataset_unprocessed[['50803','50804']].apply(lambda x: str(x))['50803']

'hadm_id\n100001    0\n100003    0\n100006    0\n100007    0\n100009    0\n         ..\n199993    0\n199994    0\n199995    0\n199998    0\n199999    0\nName: 50803, Length: 54871, dtype: object'

In [99]:
df_final_dataset['50803_1.0'].value_counts()

0    53804
1     1067
Name: 50803_1.0, dtype: int64

In [ ]:
df_dataset_processed = featues_datasets_all_patients.on_hot_encode_categorical(df_dataset_unprocessed)

In [27]:
tmp_df = clean_and_bin_numeric_values(df_final_dataset)

In [24]:
num_cols = select_numeric_values_to_process(df_final_dataset)

In [29]:
tmp_df[num_cols].dtypes

hadm_id                   object
50801                     object
50802                     object
50803                     object
50804                     object
                           ...  
Vancomycin Enema          object
Vancomycin HCl            object
Vancomycin Oral Liquid    object
n_admits                  object
wounds                    object
Length: 414, dtype: object

In [36]:
f_final_dataset = io.load_dataframe('df_final_dataset')

In [39]:
f_final_dataset.select_dtypes(include='number')

,hadm_id,50800_flag,50801_flag,50802_flag,50803_flag,50804_flag,50805_flag,50806_flag,50808_flag,50809_flag,...,Vancomycin,Vancomycin,Vancomycin 25mg/mL Ophth Soln,Vancomycin Antibiotic Lock,Vancomycin Desensitization,Vancomycin Enema,Vancomycin HCl,Vancomycin Oral Liquid,n_admits,wounds
0,100001,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,-1,0,-1,0,1,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100006,-1,0,-1,0,-1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100007,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100009,0,0,-1,0,-1,0,-1,-1,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54866,199993,-1,0,-1,0,-1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54867,199994,-1,0,-1,0,-1,0,0,-1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
54868,199995,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
54869,199998,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from dataproc.proc_utils import bin_numerics

In [22]:
# create a cohort dataframe and numpy file
import build_cohort_dataset
df_cohort = build_cohort_dataset.build_cohort(params, df_final_dataset)

df_labels: (4130, 2)
cohort dataset: (4085, 843)
cohort data saved to data/fulldata.npy


In [25]:
df_cohort.drop(columns=['index']).to_csv('data/fulldata.csv', sep=',', index=False)

## Tools to explore too many unique values

In [ ]:
df_dataset_unprocessed[categorical_cols].nunique().sort_values(ascending=False)[0:20]

In [ ]:
allvals = df_dataset_unprocessed['51508'].unique().tolist()
len(allvals), df_dataset_unprocessed['51508'].notna().sum()

In [ ]:
df_dataset_unprocessed['51508'].value_counts()

In [ ]:
[x for x in allvals if x and not x == np.nan and not str(x).replace('.','').isnumeric()]

In [ ]:
pd.Series(sorted([float(x) for x in allvals if x and str(x).replace('.','').isnumeric() and float(x) > 1000.0])).describe()


In [ ]:
df_dataset_unprocessed.nunique().sort_values(ascending=False)

## THE END

In [ ]:
import torch
torch.__version__

In [ ]:
fulldata